<div style="background-color: white; padding: 10px; border-bottom: 6px solid #C2172D;">
    <h2 style="color: black" id="introduction">Batch Data Processing with Apache Spark</h2>
    <p></p>
</div>


ATALAY BEKTAŞ

Not: Data'daki hatalı Retail.perquet dosyası düzeltildi


## [Contents](#contents)
1. [Introduction](#introduction)
2. [Importing the libraries](#library)
3. [Reading the data](#read_data)
4. [SparkSQL Practices](#spark_sql_practices)
   * [Selecting columns](#selecting_columns)
   * [Data manipulation](#data_manipulation)
   * [Filtering rows](#filtering_rows)
   * [Aggregating data](#aggregating_data)
   * [Joining](#joining)
5. [Case Studies](#assignments)
   * [Assignment 1: Jacket sales per region](#assignment_1)
   * [Assignment 2: Maximum turnover of the retailer regions](#assignment_2)

<div style="background-color: white; padding: 10px; border-bottom: 4px solid #C2172D;">
    <a id="introduction">
        <h3 style="color: #C2172D">1. Introduction</h3>
    </a>  
</div>

<img src="/content/drive/My Drive/yzup/huwaei assigment/data_model.svg"  style="width:1000px; padding: 20px"/>

#### SQL Tables Description
- **FactSale:** Sales transactions fact table
- **FactPurchase:** Purchases fact table
- **DimRetailer:** Retailer details dimension table
- **DimCustomer:** Customer details dimension table
- **DimProduct:** Product details dimension table
- **DimRegion:** Region details dimension table
- **DimDate:** Date dimension table
- **DimSupplier:** Supplier details dimension table

<div style="background-color: white; padding: 10px; border-bottom: 4px solid #C2172D;">
    <a id="library">
        <h3 style="color: #C2172D">2. Importing the libraries</h3>
    </a>
</div>

In [2]:
from pyspark.sql import SparkSession, functions as F
from google.colab import drive
drive.mount('/content/drive')

from IPython.display import SVG
SVG("/content/drive/My Drive/yzup/huwaei assigment/data_model.svg")

Mounted at /content/drive


<div style="background-color: white; padding: 10px; border-bottom: 4px solid #C2172D;">
    <a id="read_data">
        <h3 style="color: #C2172D">3.  Parquet Data  Okunması ve Spark SQL Görünümlerinin Oluşturulması</h3>
    </a>
</div>

In [3]:
# Creating new SparkSession instance

spark = SparkSession.builder.appName("PySparkExample").getOrCreate()


# DataFrame olarak parquet data okunması
df_pur = spark.read.parquet("/content/drive/My Drive/yzup/huwaei assigment/data/purchase")
df_sal = spark.read.parquet("/content/drive/My Drive/yzup/huwaei assigment/data/sale")
df_cus = spark.read.parquet("/content/drive/My Drive/yzup/huwaei assigment/data/customer")
df_ret = spark.read.parquet("/content/drive/My Drive/yzup/huwaei assigment/data/retailer")
df_pro = spark.read.parquet("/content/drive/My Drive/yzup/huwaei assigment/data/product")
df_sup = spark.read.parquet("/content/drive/My Drive/yzup/huwaei assigment/data/supplier")
df_reg = spark.read.parquet("/content/drive/My Drive/yzup/huwaei assigment/data/region")
df_date = spark.read.parquet("/content/drive/My Drive/yzup/huwaei assigment/data/date")

#  Spark SQL sorguları için temporary view tabloları
df_cus.createOrReplaceTempView("DimCustomer")
df_pur.createOrReplaceTempView("FactPurchase")
df_sal.createOrReplaceTempView("FactSale")
df_ret.createOrReplaceTempView("DimRetailer")
df_pro.createOrReplaceTempView("DimProduct")
df_sup.createOrReplaceTempView("DimSupplier")
df_reg.createOrReplaceTempView("DimRegion")
df_date.createOrReplaceTempView("DimDate")

<div style="background-color: white; padding: 10px; border-bottom: 4px solid #C2172D;">
    <a id="spark_sql_practices">
        <h3 style="color: #C2172D">4. Spark SQL Practices</h3>
    </a>
</div>

**<a id="selecting_columns">Selecting columns</a>**

In [4]:
# Müşteri verilerinin genel olarak incelenmesi

spark.sql("SELECT customer_id, name, surname, birth_date FROM DimCustomer LIMIT 5").show()

+-----------+-------+--------+----------+
|customer_id|   name| surname|birth_date|
+-----------+-------+--------+----------+
|          1| Jazmin|  Burril|1958-09-22|
|          2| Dalila|   Faers|2000-11-08|
|          3|Wayland|Walework|1976-03-08|
|          4|Amberly|  Haquin|1948-10-08|
|          5|Garrett|   Frear|1957-09-25|
+-----------+-------+--------+----------+



In [5]:
# DataFrame deki Müşteri verilerini genel olarak incelenmesi

df_cus.select("customer_id", "name", "surname", "birth_date").show(5)

+-----------+-------+--------+----------+
|customer_id|   name| surname|birth_date|
+-----------+-------+--------+----------+
|          1| Jazmin|  Burril|1958-09-22|
|          2| Dalila|   Faers|2000-11-08|
|          3|Wayland|Walework|1976-03-08|
|          4|Amberly|  Haquin|1948-10-08|
|          5|Garrett|   Frear|1957-09-25|
+-----------+-------+--------+----------+
only showing top 5 rows



**<a id="data_manipulation">Data manipulation: </a>** Calculating the ages from date of birth data.

In [6]:
# Yaş bilgisinin hesaplanması ve örneklerin incelenmesi

spark.sql("""
SELECT
    customer_id
    ,name
    ,surname
    ,YEAR(CURRENT_DATE()) - YEAR(birth_date) AS age
FROM DimCustomer
LIMIT 5
""").show()

+-----------+-------+--------+---+
|customer_id|   name| surname|age|
+-----------+-------+--------+---+
|          1| Jazmin|  Burril| 67|
|          2| Dalila|   Faers| 25|
|          3|Wayland|Walework| 49|
|          4|Amberly|  Haquin| 77|
|          5|Garrett|   Frear| 68|
+-----------+-------+--------+---+



In [9]:
#df_cus DataFrame ine yaş hesaplayan bir sütun ekler. df_cus müsteri data frame üzerinden yaş hesaplanması ve örneklerin incelenmesi
(
    df_cus.withColumn("age", F.year(F.current_date()) - F.year("birth_date"))
    .select("customer_id", "name", "surname", "age")
    .show(5)
)

+-----------+-------+--------+---+
|customer_id|   name| surname|age|
+-----------+-------+--------+---+
|          1| Jazmin|  Burril| 67|
|          2| Dalila|   Faers| 25|
|          3|Wayland|Walework| 49|
|          4|Amberly|  Haquin| 77|
|          5|Garrett|   Frear| 68|
+-----------+-------+--------+---+
only showing top 5 rows



**<a id="filtering_rows">Filtering rows</a>**

In [10]:
# 30 yaşın üzerindeki müsterilerin filtrelenerek incelenmesi

spark.sql("""
SELECT
    name
    ,surname
    ,age
FROM
(
    SELECT
        customer_id
        ,name
        ,surname
        ,YEAR(CURRENT_DATE()) - YEAR(birth_date) AS age
    FROM DimCustomer
)
WHERE age >= 30
LIMIT 5
""").show()

+-------+--------+---+
|   name| surname|age|
+-------+--------+---+
| Jazmin|  Burril| 67|
|Wayland|Walework| 49|
|Amberly|  Haquin| 77|
|Garrett|   Frear| 68|
|  Horst|   Isted| 50|
+-------+--------+---+



In [12]:
# df_cus DataFrame ine yaş sütunu ekler  yaş grubuna göre müşteri verilerini incelenmesi

(
    df_cus.withColumn("age", F.year(F.current_date()) - F.year("birth_date"))
    .select("name", "surname", "age")
    .filter(F.col("age") >= 30)
    .show(5)
)

+-------+--------+---+
|   name| surname|age|
+-------+--------+---+
| Jazmin|  Burril| 67|
|Wayland|Walework| 49|
|Amberly|  Haquin| 77|
|Garrett|   Frear| 68|
|  Horst|   Isted| 50|
+-------+--------+---+
only showing top 5 rows



**<a id="aggregating_data">Aggregating data</a>**

In [13]:

spark.sql("""
SELECT
    order_id
    ,SUM(quantity) AS total_quantity
    ,SUM(total_amt) AS total_amount
FROM FactSale
GROUP BY order_id
ORDER BY total_quantity DESC
LIMIT 10
""").show()

+--------+--------------+------------+
|order_id|total_quantity|total_amount|
+--------+--------------+------------+
|    3647|            13|         521|
|    2574|            13|         488|
|    3515|            13|         402|
|     101|            12|         359|
|     440|            12|         426|
|    3763|            12|         323|
|    1585|            12|         488|
|    3289|            12|         327|
|    1382|            11|         452|
|    1752|            11|         298|
+--------+--------------+------------+



In [14]:

(
    df_sal.groupBy("order_id").agg(
        F.sum("quantity").alias("total_quantity"),
        F.sum("total_amt").alias("total_amount")
    ).orderBy("total_quantity", ascending=False)
    .show(10)
)

+--------+--------------+------------+
|order_id|total_quantity|total_amount|
+--------+--------------+------------+
|    3647|            13|         521|
|    2574|            13|         488|
|    3515|            13|         402|
|     101|            12|         359|
|     440|            12|         426|
|    3763|            12|         323|
|    1585|            12|         488|
|    3289|            12|         327|
|    2337|            11|         357|
|    3743|            11|         359|
+--------+--------------+------------+
only showing top 10 rows



**<a id="joining">Joining</a>**

| JOIN Türü                            | Açıklama  |
|--------------------------------------|----------|
| **INNER JOIN**                       | Eşleşen kayıtları getirir. |
| **LEFT JOIN**                        | Sol tablodaki tüm kayıtları, sağdan eşleşenlerle birlikte getirir. |
| **RIGHT JOIN**                       | Sağ tablodaki tüm kayıtları, soldan eşleşenlerle birlikte getirir. |
| **FULL JOIN**                        | Her iki tablodaki tüm kayıtları getirir, eşleşmeyenlere NULL yazar. |
| **LEFT OUTER JOIN (ya da LEFT JOIN)** | Sol tablodaki tüm kayıtları döndürür; sağda eşleşmeyenler için NULL koyar. |
| **RIGHT OUTER JOIN (ya da RIGHT JOIN)** | Sağ tablodaki tüm kayıtları döndürür; solda eşleşmeyenler için NULL koyar. |
| **FULL OUTER JOIN (ya da OUTER JOIN)** | Her iki tablodaki tüm kayıtları döndürür; eşleşmeyen yerler NULL olur. |
| **CROSS JOIN**                        | İki tablonun tüm kayıtlarını çarpar (kartaizyen çarpım); filtre olmadan tüm kombinasyonlar döner. Çok büyük sonuçlar üretebilir. |


In [15]:
# Bölgelere göre müşteri yaşı incelenmesi

spark.sql("""
SELECT
    region_name
    ,AVG(YEAR(CURRENT_DATE()) - YEAR(birth_date)) AS age
FROM DimCustomer cus
INNER JOIN DimRegion reg
ON cus.city_id = reg.city_id
GROUP BY region_name
ORDER BY age DESC
""").show()

+-----------------+------------------+
|      region_name|               age|
+-----------------+------------------+
|          Akdeniz| 51.81521739130435|
|     Dogu Anadolu| 51.13095238095238|
|Guneydogu Anadolu| 49.58119658119658|
|          Marmara|49.189542483660134|
|       Ic Anadolu| 49.07772020725388|
|        Karadeniz| 48.75121951219512|
|              Ege|47.888888888888886|
+-----------------+------------------+



In [16]:
(
    df_cus
    .join(df_reg, df_cus.city_id == df_reg.city_id)
    .groupBy("region_name").agg(
        F.avg(F.year(F.current_date()) - F.year("birth_date")).alias("age")
    )
    .orderBy("age", ascending=False)
    .show()
)

+-----------------+------------------+
|      region_name|               age|
+-----------------+------------------+
|          Akdeniz| 51.81521739130435|
|     Dogu Anadolu| 51.13095238095238|
|Guneydogu Anadolu| 49.58119658119658|
|          Marmara|49.189542483660134|
|       Ic Anadolu| 49.07772020725388|
|        Karadeniz| 48.75121951219512|
|              Ege|47.888888888888886|
+-----------------+------------------+



Yıl içinde aylık stışların incelenmesi

In [17]:
df_sal.withColumn("month", F.month("date")) \
    .groupBy("month") \
    .agg(F.sum("total_amt").alias("monthly_revenue")) \
    .orderBy("month") \
    .show()

+-----+---------------+
|month|monthly_revenue|
+-----+---------------+
|    1|          62906|
|    2|          53313|
|    3|          62365|
|    4|          65533|
|    5|          61612|
|    6|          53981|
|    7|          62887|
|    8|          54599|
|    9|          57765|
|   10|          60040|
|   11|          58401|
|   12|          55693|
+-----+---------------+



Bölgelere göre ürün satışı incelenmesi


In [18]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

top_product_df = df_sal \
    .join(df_cus, "customer_id") \
    .join(df_reg, df_cus.city_id == df_reg.city_id) \
    .join(df_pro, "product_id") \
    .groupBy("region_name", "product_type") \
    .agg(F.sum("quantity").alias("total_quantity"))

window_spec = Window.partitionBy("region_name").orderBy(F.desc("total_quantity"))

top_product_df \
    .withColumn("rank", row_number().over(window_spec)) \
    .filter("rank == 1") \
    .select("region_name", "product_type", "total_quantity") \
    .show()

+-----------------+------------+--------------+
|      region_name|product_type|total_quantity|
+-----------------+------------+--------------+
|          Akdeniz|    Trousers|           577|
|     Dogu Anadolu|      Jacket|          1068|
|              Ege|      Tshirt|           458|
|Guneydogu Anadolu|      Jacket|           723|
|       Ic Anadolu|      Tshirt|          1184|
|        Karadeniz|    Trousers|          1188|
|          Marmara|      Tshirt|           970|
+-----------------+------------+--------------+



<div style="background-color: white; padding: 10px; border-bottom: 4px solid #C2172D;">
    <a id="case_studies">
        <h3 style="color: #C2172D">5. Case Studies</h3>
    </a>  
</div>

<div style="background-color: white; padding: 10px;">
    <a id="assignment_1">
        <h4 style="color: #0D9276">Assignment 1: Jacket sales per region</h3>
    </a>
</div>
<br>
<h4>
    Write SparkSQL scripts that results: Region-based total quantity and amount of jacket sales between June and August 2023.
</h4>
<p>The expected out is as follows: </p>

| region_name       | product_type | total_quantity | total_amount |   |
|-------------------|--------------|----------------|--------------|---|
| Marmara           | Jacket       | 213            | 8358         |   |
| Dogu Anadolu      | Jacket       | 284            | 11547        |   |
| Guneydogu Anadolu | Jacket       | 176            | 6981         |   |
| Ic Anadolu        | Jacket       | 260            | 10496        |   |
| Akdeniz           | Jacket       | 162            | 6637         |   |
| Karadeniz         | Jacket       | 310            | 12582        |   |
| Ege               | Jacket       | 101            | 3953      


### External links
- https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.join.html
- https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.filter.html
- https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.groupBy.html   |   |

In [19]:
# Your Spark SQL Solution:

#joined FactSale with  DimProduct DimCustomer and DimRegion tables
# Summed Jacket sales between June 1 2023 and August 31 2023 then


spark.sql("""
    SELECT
        DimRegion.region_name,
        DimProduct.product_type,
        SUM(FactSale.quantity) AS total_quantity,
        SUM(FactSale.total_amt) AS total_amount
    FROM FactSale
    INNER JOIN DimProduct ON FactSale.product_id = DimProduct.product_id
    INNER JOIN DimCustomer ON FactSale.customer_id = DimCustomer.customer_id
    INNER JOIN DimRegion ON DimCustomer.city_id = DimRegion.city_id
    WHERE
        DimProduct.product_type = 'Jacket'
        AND FactSale.date BETWEEN '2023-06-01' AND '2023-08-31'
    GROUP BY
        DimRegion.region_name,
        DimProduct.product_type
""").show()



+-----------------+------------+--------------+------------+
|      region_name|product_type|total_quantity|total_amount|
+-----------------+------------+--------------+------------+
|          Marmara|      Jacket|           213|        8358|
|     Dogu Anadolu|      Jacket|           284|       11547|
|Guneydogu Anadolu|      Jacket|           176|        6981|
|       Ic Anadolu|      Jacket|           260|       10496|
|          Akdeniz|      Jacket|           162|        6637|
|        Karadeniz|      Jacket|           310|       12582|
|              Ege|      Jacket|           101|        3953|
+-----------------+------------+--------------+------------+



In [20]:
# Your PySpark Solution:

df_sal.join(df_pro, df_sal.product_id == df_pro.product_id, "inner") \
    .join(df_cus, df_sal.customer_id == df_cus.customer_id, "inner") \
    .join(df_reg, df_cus.city_id == df_reg.city_id, "inner") \
    .filter((df_pro.product_type == "Jacket") &
            (df_sal.date.between("2023-06-01", "2023-08-31"))) \
    .groupBy(df_reg.region_name, df_pro.product_type) \
    .agg(
        F.sum(df_sal.quantity).alias("total_quantity"),
        F.sum(df_sal.total_amt).alias("total_amount")
    ) \
    .show()


+-----------------+------------+--------------+------------+
|      region_name|product_type|total_quantity|total_amount|
+-----------------+------------+--------------+------------+
|          Marmara|      Jacket|           213|        8358|
|     Dogu Anadolu|      Jacket|           284|       11547|
|Guneydogu Anadolu|      Jacket|           176|        6981|
|       Ic Anadolu|      Jacket|           260|       10496|
|          Akdeniz|      Jacket|           162|        6637|
|        Karadeniz|      Jacket|           310|       12582|
|              Ege|      Jacket|           101|        3953|
+-----------------+------------+--------------+------------+



<div style="background-color: white; padding: 10px;">
    <a id="assignment_2">
        <h4 style="color: #0D9276">Assignment 2: Maximum turnover of the retailer regions</h3>
    </a>
</div>
<br>
<h4>
    Find the maximum turnover region of each retailer, and obtain total amount for each retailer and region.
</h4>
<p>The expected out is as follows: </p>

| retailer_id | retailer_name | region_name | total_amount |
|-------------|---------------|-------------|--------------|
| 1           | A             | Karadeniz   | 42642        |
| 2           | B             | Ic Anadolu  | 71689        |
| 3           | C             | Ic Anadolu  | 11995        |
| 4           | D             | Karadeniz   | 16081        |

<br>

In [21]:
# Your Spark SQL Solution:
# max total_amount region of each retailer
# city_id values in the DimRetailer table represent the location of the retailer, not the place where the products are sold
# using  city_id in the DimCustomer table to determine sale regions

# FactSale, DimCustomer, DimRegion, and DimRetailer joined on customer_id, city_id, and retailer_id, and the region with the highest sales amount for each retailer was selected
# selected highest region total_amount for each retailer

spark.sql("""
WITH RetailerRegionSales AS (
    SELECT
        FactSale.retailer_id,
        DimRetailer.retailer_name,
        DimRegion.region_name,
        SUM(FactSale.total_amt) AS total_amount
    FROM FactSale
    INNER JOIN DimCustomer ON FactSale.customer_id = DimCustomer.customer_id
    INNER JOIN DimRegion ON DimCustomer.city_id = DimRegion.city_id
    INNER JOIN DimRetailer ON FactSale.retailer_id = DimRetailer.retailer_id
    GROUP BY FactSale.retailer_id, DimRetailer.retailer_name, DimRegion.region_name
)
SELECT
    RetailerRegionSales.retailer_id,
    RetailerRegionSales.retailer_name,
    RetailerRegionSales.region_name,
    RetailerRegionSales.total_amount
FROM RetailerRegionSales
WHERE (RetailerRegionSales.retailer_id, RetailerRegionSales.total_amount) IN (
    SELECT retailer_id, MAX(total_amount)
    FROM RetailerRegionSales
    GROUP BY retailer_id
)
ORDER BY RetailerRegionSales.retailer_id

""").show()

+-----------+-------------+-----------+------------+
|retailer_id|retailer_name|region_name|total_amount|
+-----------+-------------+-----------+------------+
|          1|            A|  Karadeniz|       42642|
|          2|            B| Ic Anadolu|       71689|
|          3|            C| Ic Anadolu|       11995|
|          4|            D|  Karadeniz|       16081|
+-----------+-------------+-----------+------------+



In [22]:
#Your PySpark Solution:

df_sal.join(df_cus, df_sal.customer_id == df_cus.customer_id) \
    .join(df_reg, df_cus.city_id == df_reg.city_id) \
    .join(df_ret, df_sal.retailer_id == df_ret.retailer_id) \
    .groupBy(df_sal.retailer_id, df_ret.retailer_name, df_reg.region_name) \
    .agg(F.sum(df_sal.total_amt).alias("total_amount")) \
    .orderBy("retailer_id") \
    .orderBy(F.desc("total_amount")) \
    .dropDuplicates(["retailer_id"]) \
    .show()


+-----------+-------------+-----------+------------+
|retailer_id|retailer_name|region_name|total_amount|
+-----------+-------------+-----------+------------+
|          1|            A|  Karadeniz|       42642|
|          2|            B| Ic Anadolu|       71689|
|          3|            C| Ic Anadolu|       11995|
|          4|            D|  Karadeniz|       16081|
+-----------+-------------+-----------+------------+

